# Prompt Optimization and Evaluation with OpenAI API

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>


## Description:

This app utilizes OpenAI's GPT-4 model to optimize prompts and evaluate their effectiveness. The PromptOptimizer class refines a given prompt using instructions, iterating several times to improve clarity, conciseness, and effectiveness. After optimization, the app evaluates the original and optimized prompts to generate a comparison report with reasons and scores. The app can be used to optimize prompts for production environments, ensuring they are effective and clear for LLM usage.

## Step 1: Installing Requirements  

The code defines the install_requirements function to install packages from `requirements.txt` if they aren't already installed. It retries up to three times if the installation fails, exiting with an error if all attempts fail. This ensures the required dependencies are in place before continuing.


In [40]:
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return

install_requirements()

## Step 2: Setting Up Environment Variables  

The code loads environment variables using `load_dotenv()` to ensure sensitive information like API keys is available. 

It then checks if required variables, such as `OPENAI_API_KEY`, are set, prompting the user to set them if missing and exiting the program.

In [42]:
from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""
    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")
    load_dotenv()

    variables_to_check = ["OPENAI_API_KEY"]

    for var in variables_to_check:
        check_env(var)

setup_env()

## Step 3: Prompt Optimization and Evaluation with OpenAI

The PromptOptimizer class refines user prompts for OpenAI’s GPT-4 model by optimizing them for clarity and effectiveness over multiple iterations. 

It uses the OpenAI API to send the prompt with optimization instructions, refining it until the desired result is achieved. 

The optimize method runs the optimization process, while the get_optimized_prompt method retrieves the final version. 

The evaluate method compares the original and optimized prompts, generating a markdown report with a detailed comparison. 

This class ensures the prompt is concise, clear, and production-ready.


In [44]:
from openai import OpenAI
import os


class PromptOptimizer:
    """A simple prompt optimization class that uses OpenAI's Chat API to optimize prompts."""

    def __init__(self, prompt: str):
        """Initializes the PromptOptimizer with the given prompt. """
        self.ai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        self.prompt = prompt
        self.optimized_prompt = None

    def optimize(self, optimize_instructions = [
        "Refine the given prompt so that it is fit for use in production environments.",
        "Make the prompt concise, detailed and clear with a good balance of effectivenss and usefulness.",
    ], turns = 3) -> str:
        """Optimizes the prompt based on optimize instructions. """

        prompt_to_optimize = self.prompt
        previous_optimized_prompt = prompt_to_optimize
        for i in range(turns):
            try:
                print(f"Optimizing prompt - Turn {i+1}...")
                optimize_prompt = f"""
                    You will be given a prompt, refine it as per the given instructions. 
                    In addition to the instructions, come up with improvements and apply the refinements to the prompt.
                    Respond only with the final refined prompt after all improvements are made.
                    Instructions: ```{"\n".join(optimize_instructions)}```
                    Prompt: {previous_optimized_prompt}
                """
                system = "You are an AI prompt optimizer with deep understanding of LLM technology."
                ai_response = self.ai.chat.completions.create(
                    messages=[
                        {"role": "system", "content": system},
                        { "role": "user", "content": optimize_prompt }
                    ],
                    model="gpt-4o-mini"
                )
                previous_optimized_prompt = ai_response.choices[0].message.content
            except Exception as e:
                print(f"Failed to optimize prompt for turn {i+1}: {e}")
                continue
        self.optimized_prompt = previous_optimized_prompt
        return self.optimized_prompt
    
    def get_optimized_prompt(self) -> str:
        """Returns the optimized prompt."""
        if not self.optimized_prompt:
            optimized_prompt = self.optimize()
            if not optimized_prompt:
                print("Failed to optimize the prompt.")
        return self.optimized_prompt
    

    def evaluate(self):
        try:
            print("Evaluating the optimized prompt...")
            optimized_prompt = self.get_optimized_prompt()

            if not optimized_prompt:
                print("Failed to get the optimized prompt, can't evaluate.")
                return None
            
            evaluation_prompt = f"""
                    Given two prompts, compare the original prompt and the optimized prompt. 
                    Provide a table with the comparison of the two prompts along with reasons and scores.
                    In the markdown response provide a title, summary, and conclusion and the table. 
                    Respond in markdown format strictly. 
                    Make sure the markdown tables are compatible with Jupyter notebooks.
                    Original Prompt: {self.prompt}
                    Optimized Prompt: {optimized_prompt}
                """
            
            response = self.ai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are expert in prompt evaluation."},
                    {"role": "user", "content": evaluation_prompt}
                ])
            
            header = f"""# Prompt Optimization Report\n## Original Prompt\n{self.prompt}\n## Optimized Prompt\n{self.optimized_prompt}"""
            report = response.choices[0].message.content
            full_report = header + report
            return full_report
        except Exception as e:
            print(f"Failed to evaluate the optimized prompt: {e}")
            return None

## Step 4: Run Prompt Optimization and Generate Evaluation Report

The run_prompt_optimizer function optimizes a given prompt using the PromptOptimizer class and generates an evaluation report.

 It creates an instance of the PromptOptimizer with the provided prompt, calls get_optimized_prompt() to retrieve the optimized prompt, and clears the notebook output for a clean display. 
 
 It then evaluates the original and optimized prompts using evaluate() and returns the evaluation report in Markdown format for proper rendering in Jupyter notebooks.


In [45]:
from IPython.display import clear_output, Markdown

def run_prompt_optimizer(prompt: str) -> None:
    prompt_optimizer = PromptOptimizer(prompt)
    optimized_prompt = prompt_optimizer.get_optimized_prompt()
    clear_output()
    report = prompt_optimizer.evaluate()
    return Markdown(report)

## Step 5: Run Prompt Optimization and Evaluation

### Example 1:

The prompt asks for a detailed plan for learning Python programming in 2025.

`run_prompt_optimizer(prompt)` will optimize this prompt and generate an evaluation report comparing the original and optimized versions.

### Example 2:

The prompt asks for the Standard Operating Procedure (SOP) for a conflict resolution strategy for a team of 5 members in a Big 4 firm.

`run_prompt_optimizer(prompt)` will optimize this prompt and provide an evaluation report as well.

In both examples, the function optimizes and evaluates the prompts using the `PromptOptimizer` class.


In [ ]:
# Example 1

prompt = "Give me a detailed plan for learning Python Programming in 2025."

run_prompt_optimizer(prompt)

# Example 2

prompt = "Give me the SOP of a conflict resolution strategy for a team of 5 members in a Big 4 Firm."

run_prompt_optimizer(prompt)


## Conclusion:

This app provides an easy-to-use interface for refining prompts and evaluating their effectiveness in AI-driven environments. By using OpenAI's GPT-4 model, the app ensures that prompts are optimized to be concise, clear, and useful, followed by a detailed evaluation report comparing the original and optimized prompts.

---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>